# Map Reduce Paradigm  

This project is about one of the core strategies in distributed processing: divide and conquer. Word counting from the text of _Alice in Wonderland_ has been used to illustrate the difference between a scalable and non-scalable algorithm, by using Python and Bash scripting. This projects helps understand folowing concepts:
* ... the __Bias-Variance tradeoff__ as it applies to Machine Learning.
* ... why word counting is considered to be an __"Embarrassingly Parallel"__ task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.

# Notebook Set-Up

In [3]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)

In [4]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [3]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

mkdir: cannot create directory `data': File exists


# Bias - Variance
__Bias-variance trade off explained below along with what it means to "decompose" sources of error and how is this used in machine learning?__

> High bias is underfitting, High variance is overfitting(the model might be learning from the noise too) for e.g the regression output will be a curve that would fit data perfectly. 
To Optimize the error in our model, we need to find the right balance between bias and variance. This is called Bias-Variance Tradeoff. Train the model in such a way that it gives almost the same error on both train and test data.
"decompose" sources of error is breaking down types of error into reducible and irreducible errors.
it is possible to show that the expected test MSE, for a given value x0, can always be decomposed into the sum of three fundamental quantities: 

1) the variance of $\hat{f}(x0)$, 

2) the squared bias of $\hat{f}(x0)$ and, 

3) the variance of the error term ϵ (irreducible error)

$ Err(x) = \big(E[\hat{f}(x)] - f(x)\big)^2 + E\big[\big(\hat{f}(x)-E[\hat{f}(x)]\big)^2\big] + \sigma^2_e $

or simply,

$ Err(x) = Bias^2 + Variance + Irreducible Error $

# Tokenizing
A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. 

* There can be several scenarios where a tokenizer, if setup in a diferent way will give different results

1)If we decide to consider punctuation symbol while extracting words from the text file, then it will lead to a different result as compared to, if punctuation symbols were not considered. 

2)There can be another case, where upper and lower cases can be tokenized as two different words. 

3)There can be another scenario where singular and plural words can be a considered as 2 separate tokens or words

In [7]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile(r'[a-z]+')  

In [8]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data

In [9]:
# Download Full text 
# replace 'curl' with 'wget' or equivalent command of your choice.

!curl "https://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0   149k      0  0:00:01  0:00:01 --:--:--  149k


In [10]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt


The jupyter magic command `%%writefile` is a convenient way to develop a habit of creating small files with simulated data for use in developing, debugging and testing code. 
__The following cells can be run to create a test data file for use in word counting task.__

In [11]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [12]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt
alice_test.txt.output


# Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

In [13]:
# part a - code is available in wordCount.py

In [14]:
# part b - run it to test your script
!python wordCount.py < data/alice_test.txt

In [15]:
# part c - run it as is to perform the word count. In the same cell, pipe the output to file

!python wordCount.py < data/alice.txt > data/alice_counts.txt


Take a look at the first 10 words & their counts.

In [16]:
!head data/alice_counts.txt

__Check your results:__ How many times does the word "alice" appear in the book? 

In [17]:
!grep alice data/alice_counts.txt

__Check your results:__ How many times does the word "hatter" appear in the book? 

In [18]:
!grep hatter data/alice_counts.txt

__Check your results:__ How many times does the word "queen" appear in the book? 

In [19]:
!grep queen data/alice_counts.txt

# Unix Sorting 
Another common task is to make strategic use of sorting.     

Few important points to note about sorting:

* a) the Big O complexity of the fastest comparison based sorting algorithms? 

* b) the default sorting algorithm in MapReduce is MergeSort algorithm. Big O complexity of this algorithm is: Time complexity is 𝑂(𝑛𝑙𝑜𝑔𝑛) and Space complexity is 𝑂(𝑛) 

* c) the reason why this algorithm was chosen is because MapReduce implements sorting algorithm to automatically sort the output key-value pairs from the mapper by their keys. One cannot change the MapReduce sorting method, the reason is that data comes from the different nodes to a single point, so the best algorithm that can be used here is the merge sort
 

In [17]:
# unix command to check how many records are in word count file
!wc -l data/alice_counts.txt

3006 data/alice_counts.txt


In [18]:
# unix command to sort word counts alphabetically 
!sort data/alice_counts.txt > data/alice_counts_A-Z.txt

In [19]:
# run to confirm if sort worked
!head data/alice_counts_A-Z.txt   

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [20]:
# unix command to sort word counts from highest to lowest count
!sort -nr -k 2 data/alice_counts.txt > data/alice_counts_sorted.txt

In [21]:
# part e - run to confirm if sort worked
!head data/alice_counts_sorted.txt  

the	1839
and	942
to	811
a	695
of	638
it	610
she	553
i	546
you	486
said	462


# Parallel Word Count (part 1)

What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available, it would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. 
> __In this exercise we will use a bash script to "parallelize" Word Count.__


In [22]:
# make sure your scripts are executable
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [23]:
# The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. 
# parallel word count on Alice text on 4 parallel processes. Results redirected into a file called `alice_pCounts.txt.

!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'



> The count for word 'alice' does not come as expected. Here, each of the mappers are outputting the count of their chunk.
In order to fix the problem, we need to add the reducer that will aggregate the results from the mappers.

In [24]:
# check alice count 
!grep alice data/alice_pCounts.txt

alice	113
alice	126
alice	122
alice	42


In [25]:
# make sure the aggregateCounts script is executable
!chmod a+x aggregateCounts_v1.py

> The python script, `aggregateCounts_v1.py`, reads word counts from standard input and combines any duplicates it encounters. In `pWordCount_v1.sh`, a one-line modification has been made so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts.

In [20]:
# parallel word count on Alice text
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [21]:
# check alice count
!grep alice data/alice_pCounts.txt

alice	403


In [23]:
# !man sort

# Parallel Word Count (part 2)

From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, there is a major scalability concern with this particular implementation. __In this exercise we'll fix our implementation of parallel word count so that it will be scalable.__

__Note:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simulate the MapReduce framework. 


* __a) The potential scalability problem with the provided implementation of `aggregateCounts_v1.py` is that the Python object that aggregates our counts (defaultdict) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook.


* __b) Fortunately, a 'strategic sort' can solve this problem. The code for this is available in `pWordCount_v2.sh` file. The code makes changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) The script `aggregateCounts_v2.py` takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. 


* __d) This rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. In case, we split the word counts in half after sorting them, and then perform the reducing on two separate machines, there can be issue of duplicate words. Same set of words can exist in both the files, that can lead to miscount of words once all the words are put together at the end.


* __e) We need to come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? Put a check in place for separation while splitting up the data on to two machines. If a word exists in reducer 1, add the second matching word to the same reducer 1. If it doesn't exist, then send that new word to the second reducer. In this way, we can ensure that the one word group won't be separated into two different reducers and thereby eliminating the duplicate.

In [29]:
# Add permissions to your new files 
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.py

In [30]:
# testing code on the test file
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3
two	1


In [31]:
# running code on the Alice file 
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [32]:
# confirm that 'alice' count is correct
!grep alice data/alice_pCounts.txt

alice	403


# Question 8: Scalability Considerations

Some Big ideas that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__


> __a)__ From a scalability perspective, the two features of an "ideal algorithm" are: 

    1) In terms of data: given twice the amount of data, the same algorithm should take at most twice as long to run, all else being equal

    2) In terms of resources: given a cluster twice the size, the same algorithm should take no more than half as long to run.

> __b)__ with simple algorithm, the ideal algorithm criteria matches. With increase in the number of partition, the execution time cuts down in half pretty much. 

> __c)__ In case of word count algorithm, Ideal algorithm criteria does not hold true. Increasing our partitions does not improve the runtime, in fact, at many partitions, it gets worse. One reason for this can be the bottle neck in merging of the partitions before being given to the reducer. Another reason can be the single reducer slowing down the computation. The communication cost can also play a major role here, as we see that with increase in the number of partitions beyond 8, the communication cost is starting to take over and dominate the total time.

> __d)__ Mapper runs faster, sorting slows things down. 


__Run the following cells to create the mapper referenced in `part b`__

In [33]:
!mkdir demo

mkdir: cannot create directory `demo': File exists


In [34]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Overwriting demo/mapper.py


In [35]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [46]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

4.83 s ± 6.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

2.77 s ± 5.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.74 s ± 4.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [39]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

758 ms ± 3.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

750 ms ± 7.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

764 ms ± 4.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

824 ms ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

926 ms ± 5.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1.12 s ± 9.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Embarrassingly Parallel


> __a)__ In parallel computing, an embarrassingly parallel workload or problem is one where little or no effort is needed to separate the problem into a number of parallel tasks. This is often the case where there is little or no dependency or need for communication between those parallel tasks, or for results between them. For example, in word counting, lines from the document can be easily split and distributed across hundreds of machines because there are no inherent dependencies between two lines.
This term describes implementation of task.

> __b)__ For any function that is associative commutative, the algorithm becomes an Embarrassingly Parallel one, because of the inherent properties associated with being associative and commutative.
A commutative problem is the one where $(a+b)=(b+a)$. An associated problem is one where $(a+b)+c=a+(b+c)$. These two properties guarantee an Embarrassingly Parallel operation since the problem can be broken down and put together in different order with no impact to the ouput.